<a href="https://colab.research.google.com/github/yudumpacin/LLM/blob/main/TicaretBakanligiQ%26ADataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Türkçe Soru-Cevap Veri Seti ile LLM - 1

## Web Scrabing ile Soru-Cevap Veri Seti Toplanması

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updat

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize Chrome WebDriver with options
# Make sure you don't pass 'chromedriver' as a positional argument if specifying options
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

In [ ]:
def extract_qa_pairs(driver):
  soup = bs(driver.page_source, 'html.parser')
  qa_pairs = []
  question_pattern = re.compile(r'(?:\d+[.-]?\s*)?(Soru:)?\s*(.*\?)')
  answer_pattern = re.compile(r'Cevap:')

  # Find all text that matches the question pattern or the answer pattern
  text_elements = soup.find_all(text=True)

  # Process text elements to extract questions and answers
  current_question = None
  for element in text_elements:
      qa = {}
      if question_pattern.search(element):
          current_question = element.strip()
      elif answer_pattern.search(element) and current_question:
          # The answer is either the current element or the next non-empty sibling's text, depending on the structure
          answer = element.strip()
          next_sibling = element.parent.next_sibling
          while not next_sibling.strip() and next_sibling:
              next_sibling = next_sibling.next_sibling
          if next_sibling and not answer_pattern.search(next_sibling):
              answer += ' ' + next_sibling.strip()
          qa["Soru"] = re.sub(r'(?:\d+[.-]?\s*)?Soru:', '', current_question)
          qa["Cevap"] = answer.replace("Cevap:","")
          current_question = None  # Reset current_question to ensure pairs are matched correctly
          qa_pairs.append(qa)
  return qa_pairs


In [ ]:
def get_qa_from_url(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # First, collect all the links you're interested in by their href attributes
    link_elements = driver.find_elements(By.XPATH, "//main//section//div//div//div//div[2]//ul//li//a[contains(@href, 'sikca-sorulan-sorular')]")
    links = [link.get_attribute('href') for link in link_elements]

    datasets = pd.DataFrame()

    for link in links:
        dataset = pd.DataFrame()
        try:
            # Navigate directly to the link URL
            driver.get(link)
            sleep(5)

            # Extract data from the new page
            qa_pairs = extract_qa_pairs(driver)
            dataset = pd.DataFrame(qa_pairs)

        except Exception as e:
            print(f"Error navigating to link: {e}")
        finally:
            datasets = pd.concat([datasets, dataset])
            # Navigate back to the main page to continue the loop
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    return datasets

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
url = "https://ticaret.gov.tr/gumruk-islemleri/sikca-sorulan-sorular/bireysel"
bireysel_qas =  get_qa_from_url(url)

<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object has no attribute 'strip'


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object is not callable


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object is not callable


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object is not callable


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


In [ ]:
bireysel_qas.head()

,Soru,Cevap
0,Kanuni dayanak nedir?,"4458 sayılı Gümrük Kanununun 167 nci maddesi,..."
1,Yolcu kimdir?,"Ticaret, memuriyet, tahsil, ziyaret, tedavi v..."
2,Kimler yolcu sayılmaz?,Nakil vasıtaları sürücüleri ve hizmetlileri y...
3,Yolcu beraberi eşya nedir?,"Yolcunun beraberinde getirdiği, ticari miktar..."
4,Yolcu beraberi kişisel eşya muafiyeti nedir?,"2009/15481 sayılı Kararın ""Kişisel eşya"" başl..."


In [ ]:
bireysel_qas.shape

(69, 2)

In [ ]:
bireysel_qas.duplicated().sum()

0

In [ ]:
bireysel_qas[0:10]

,Soru,Cevap
0,Kanuni dayanak nedir?,"4458 sayılı Gümrük Kanununun 167 nci maddesi,..."
1,Yolcu kimdir?,"Ticaret, memuriyet, tahsil, ziyaret, tedavi v..."
2,Kimler yolcu sayılmaz?,Nakil vasıtaları sürücüleri ve hizmetlileri y...
3,Yolcu beraberi eşya nedir?,"Yolcunun beraberinde getirdiği, ticari miktar..."
4,Yolcu beraberi kişisel eşya muafiyeti nedir?,"2009/15481 sayılı Kararın ""Kişisel eşya"" başl..."
5,Yolcu beraberinde getirilen Ek-9 sayılı liste...,Yurda giriş yapan yolcuların Karar eki Ek-9 l...
6,Yolcunun kullandığı ilaçlarını beraberinde ge...,Ülkemize gelen yolcuların beraberlerinde ya d...
7,Yolcu beraberinde hangi evcil hayvanlar getir...,Yolcunun beraberinde olmak kayıt ve şartıyla ...
8,Yolcu Beraberinde Getirilen Cep Telefonları Ve...,"Evet vergiden muaftır. Ancak, Maliye Bakanlığ..."
9,Yolcu Beraberinde Getirilen Cep Telefonlarınd...,"Söz konusu harcın, Bilgi Teknolojileri ve İle..."


In [ ]:
bireysel_qas[-10:]

,Soru,Cevap
16,Başvuru sırasında aranacak belgeler nelerdir?,"(1) Kişilerin kendisi, eşi veya Gümrük Kanunu..."
17,Konuyla ilgili mevzuata nasıl ulaşabilirim?,7/10/2009 tarih ve 27369 (1. Mükerrer) sayılı...
18,Bedelsiz ithalat yoluyla galen araçlarda mali...,Aracın motor hacmine göre değişen oranda Özel...
0,Doğal afetler nedeniyle / kriz hallerinde yur...,4458 sayılı Gümrük Kanununun gümrük vergileri...
1,Kimler bu muafiyetten yararlanabilir?,Kriz hallerinde zarar görenlere gönderilecek ...
2,Eşyaya ilişkin şartlar nelerdir?,"Bu kapsamdaki eşya ancak,"
3,Gerçekleştirilmesi gereken gümrük işlemleri n...,Kriz hali nedeniyle yurt dışından gelen eşyay...
4,Eşyanın devri veya satışına ilişkin kural ve ...,Gümrük vergilerinden muaf olarak serbest dola...
0,Gümrüksüz satış mağazası nedir?,"Gümrüksüz satış mağazaları, Türkiye’ye gelen ..."
1,Gümrüksüz satış mağazalarındaki satış limitle...,Gümrüksüz satış mağazalarında 29/9/2009 tarih...


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
ticari_qas =  get_qa_from_url("https://ticaret.gov.tr/gumruk-islemleri/sikca-sorulan-sorular/ticari")

<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object has no attribute 'strip'


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object has no attribute 'strip'


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


...

Error navigating to link: 'NoneType' object has no attribute 'strip'


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object has no attribute 'strip'


<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)
<ipython-input-27-d5ce32808a46>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_elements = soup.find_all(text=True)


Error navigating to link: 'NoneType' object has no attribute 'strip'


In [ ]:
ticari_qas.head()

,Soru,Cevap
0,TIR karnesi kapsamında yapılan taşımalarda fa...,4458 sayılı Gümrük Kanununun bilgi ve belge t...
1,"TIR karnesinde yazılı varış ülkesi ile CMR, f...",Karayolu Taşıma Yönetmeliğinde üçüncü ülke ta...
2,Memur refakati uygulaması nasıl yapılmaktadır?,T aşınan eşyanın gümrük vergileri toplamının ...
3,TIR karnesi hamili araç şoförünün işlediği su...,“TIR Karnesi hamili” deyiminden; TIR Sözleşme...
4,TIR sisteminden geçici süreyle ihraç edilmiş ...,"Geçici ihraç kararı nedeniyle, ihracın başlad..."


In [ ]:
ticari_qas.shape

(131, 2)

In [ ]:
ticari_qas.duplicated().sum()

0

In [ ]:
all_data = pd.concat([bireysel_qas,ticari_qas])

In [ ]:
all_data.head()

,Soru,Cevap
0,Kanuni dayanak nedir?,"4458 sayılı Gümrük Kanununun 167 nci maddesi,..."
1,Yolcu kimdir?,"Ticaret, memuriyet, tahsil, ziyaret, tedavi v..."
2,Kimler yolcu sayılmaz?,Nakil vasıtaları sürücüleri ve hizmetlileri y...
3,Yolcu beraberi eşya nedir?,"Yolcunun beraberinde getirdiği, ticari miktar..."
4,Yolcu beraberi kişisel eşya muafiyeti nedir?,"2009/15481 sayılı Kararın ""Kişisel eşya"" başl..."


In [ ]:
all_data.shape

(200, 2)

In [ ]:
all_data.to_csv("ticaret_bakanligi_sss.csv")